In [1]:
import os
import sys
import random
import argparse
import numpy as np
from PIL import Image, ImageFile
import math

In [2]:
__version__ = '0.3.0'
__file__ = ''

In [3]:
image_path = "C:\\Users\\admin\\Documents\\인공지능 교육\\final_project\\Helmet Synthetic\\no_helmet\\"
image_path

'C:\\Users\\admin\\Documents\\인공지능 교육\\final_project\\Helmet Synthetic\\no_helmet\\'

In [4]:
def create_helmet(image_path):
    pic_path = image_path
    helmet_path = "white_helmet.PNG"
    show = False
    model = "hog"
    Put_helmet(pic_path, helmet_path, show, model).helmet()

class Put_helmet:
    KEY_FACIAL_FEATURES = ('nose_bridge', 'chin')

    def __init__(self, face_path, helmet_path, show=False, model='hog'):
        self.face_path = face_path
        self.helmet_path = helmet_path
        self.show = show
        self.model = model
        self._face_img: ImageFile = None
        self._helmet_img: ImageFile = None

    def helmet(self):
        import face_recognition
        
        face_image_np = face_recognition.load_image_file(self.face_path) # 얼굴 이미지 좌표값나열
        face_locations = face_recognition.face_locations(face_image_np, model=self.model) # 얼굴 위치
        face_landmarks = face_recognition.face_landmarks(face_image_np, face_locations) # 얼굴 랜드마크 좌표값
        self._face_img = Image.fromarray(face_image_np)
        self._helmet_img = Image.open(self.helmet_path)

        found_face = False
        for face_landmark in face_landmarks:
            # check whether facial features meet requirement
            skip = False
            for facial_feature in self.KEY_FACIAL_FEATURES:
                if facial_feature not in face_landmark:
                    skip = True
                    break
            if skip:
                continue

            # helmet face
            found_face = True
            self._helmet_face(face_landmark)

        if found_face:
            if self.show:
                self._face_img.show()

            # save
            self._save()
        else:
            print('Found no face.')
    
    @staticmethod
    def distance_point(point1, point2):
        con = math.sqrt((point1[0]-point2[0])**2+(point1[1]-point2[1])**2)
    
        return con
    
    @staticmethod
    def inner_division_point(point1, point2, m, n):
        con_x = (m*point2[0]+n*point1[0])/(m+n)
        con_y = (m*point2[1]+n*point1[1])/(m+n)
    
        return int(con_x), int(con_y)    

    def _helmet_face(self, face_landmark: dict):
        nose_bridge = face_landmark['nose_bridge']
        nose_point = nose_bridge[0]
        nose_top = nose_bridge[len(nose_bridge)-1]
        nose_v = np.array(nose_point)

        chin = face_landmark['chin']
        chin_len = len(chin)
        chin_bottom_point = chin[chin_len // 2]
        chin_bottom_v = np.array(chin_bottom_point)
        chin_left_point = chin[0]
        chin_right_point = chin[chin_len - 1]
        
        # get middle point
        left_eyebrow = face_landmark['left_eyebrow']
        left_eyebrow_point =  left_eyebrow[0]
        
        rignt_eyebrow = face_landmark['right_eyebrow']
        right_eyebrow_point = rignt_eyebrow[len(rignt_eyebrow)-1]
        
        point_m = [(left_eyebrow_point[0]+right_eyebrow_point[0])//2, (left_eyebrow_point[1]+right_eyebrow_point[1])//2]        

        # split helmet and resize
        width = self._helmet_img.width
        height = self._helmet_img.height
        width_ratio = 1.5
        new_height = int(np.linalg.norm(nose_v - chin_bottom_v))

        # left
        helmet_left_img = self._helmet_img.crop((0, 0, width // 2, height)) # 마스크 왼쪽 반으로 자름
        helmet_left_width = self.get_distance_from_point_to_line(chin_left_point, nose_point, chin_bottom_point)
        helmet_left_width = int(helmet_left_width * width_ratio)
        helmet_left_img = helmet_left_img.resize((helmet_left_width, new_height))

        # right
        helmet_right_img = self._helmet_img.crop((width // 2, 0, width, height))
        helmet_right_width = self.get_distance_from_point_to_line(chin_right_point, nose_point, chin_bottom_point)
        helmet_right_width = int(helmet_right_width * width_ratio)
        helmet_right_img = helmet_right_img.resize((helmet_right_width, new_height))

        # merge helmet
        size = (helmet_left_img.width + helmet_right_img.width, new_height)
        helmet_img = Image.new('RGBA', size)
        helmet_img.paste(helmet_left_img, (0, 0), helmet_left_img)
        helmet_img.paste(helmet_right_img, (helmet_left_img.width, 0), helmet_right_img)

        # rotate helmet
        angle = np.arctan2(left_eyebrow_point[1] - right_eyebrow_point[1], right_eyebrow_point[0] - left_eyebrow_point[0])
        angle *= 180 / np.pi
        rotated_helmet_img = helmet_img.rotate(angle, expand=True)
        
        offset = helmet_img.width // 2 - helmet_left_img.width
        radian = angle * np.pi / 180
        
        temp_x = 0
        temp_y = -(new_height//2)

        # calculate helmet location
        center_x = int((math.cos(-radian)*temp_x - math.sin(-radian)*temp_y) + point_m[0])
        center_y = int((math.sin(-radian)*temp_x + math.cos(-radian)*temp_y) + point_m[1])
            
        radian = angle
        box_x = center_x + int(offset * np.cos(radian)) - rotated_helmet_img.width // 2
        box_y = center_y + int(offset * np.sin(radian)) - rotated_helmet_img.height // 2

        # add helmet
        self._face_img.paste(rotated_helmet_img, (box_x, box_y), rotated_helmet_img)

    def _save(self):
        path_splits = self.face_path.split('\\')
        new_face_path = 'C:\\Users\\admin\\Documents\\인공지능 교육\\final_project\\Helmet Synthetic\\with_helmet2\\' + 'helmet' + path_splits[len(path_splits)-1]
        self._face_img.save(new_face_path)
        print(f'Save to {new_face_path}')

    @staticmethod
    def get_distance_from_point_to_line(point, line_point1, line_point2):
        distance = np.abs((line_point2[1] - line_point1[1]) * point[0] +
                          (line_point1[0] - line_point2[0]) * point[1] +
                          (line_point2[0] - line_point1[0]) * line_point1[1] +
                          (line_point1[1] - line_point2[1]) * line_point1[0]) / \
                   np.sqrt((line_point2[1] - line_point1[1]) * (line_point2[1] - line_point1[1]) +
                           (line_point1[0] - line_point2[0]) * (line_point1[0] - line_point2[0]))
        return int(distance)


In [5]:
if __name__ == '__main__':
    #cli()
    for i in range(0, 200):
#         create_helmet(image_path+'{}.png'.format(i))
        try:
            create_helmet(image_path+'{}.png'.format(i))
        except:
            print('Error')
            continue

Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Save to C:\Users\admin\Documents\인공지능 교육\final_project\Helmet Synthetic\with_helmet2\helmet102.png
Save to C:\Users\admin\Documents\인공지능 교육\final_project\Helmet Synthetic\with_helmet2\helmet103.png
Save to C:\Users\admin\Documents\인공지능 교육\final_project\Helmet Synthetic\with_helmet2\helmet104.png
Save to C:\Users\admin\Documents\인공지능 교육\final_project\Helmet Synthetic\with_helmet2\helmet

Save to C:\Users\admin\Documents\인공지능 교육\final_project\Helmet Synthetic\with_helmet2\helmet179.png
Save to C:\Users\admin\Documents\인공지능 교육\final_project\Helmet Synthetic\with_helmet2\helmet180.png
Save to C:\Users\admin\Documents\인공지능 교육\final_project\Helmet Synthetic\with_helmet2\helmet181.png
Save to C:\Users\admin\Documents\인공지능 교육\final_project\Helmet Synthetic\with_helmet2\helmet182.png
Save to C:\Users\admin\Documents\인공지능 교육\final_project\Helmet Synthetic\with_helmet2\helmet183.png
Save to C:\Users\admin\Documents\인공지능 교육\final_project\Helmet Synthetic\with_helmet2\helmet184.png
Save to C:\Users\admin\Documents\인공지능 교육\final_project\Helmet Synthetic\with_helmet2\helmet185.png
Save to C:\Users\admin\Documents\인공지능 교육\final_project\Helmet Synthetic\with_helmet2\helmet186.png
Save to C:\Users\admin\Documents\인공지능 교육\final_project\Helmet Synthetic\with_helmet2\helmet187.png
Save to C:\Users\admin\Documents\인공지능 교육\final_project\Helmet Synthetic\with_helmet2\helmet188.png
Save to C: